In [ ]:
#### Baixar imagens de um arquivo Excel com URLs
import os
import pandas as pd
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) #### Desabilitar avisos de segurança do requests

# Nome do arquivo Excel
excel_file = "cards_data.xlsx"
image_folder = "images"
os.makedirs(image_folder, exist_ok=True)

# Carregar o Excel
df = pd.read_excel(excel_file, usecols=["Image Name", "Image URL", "Baixado"])  # Lendo colunas relevantes
df.dropna(inplace=True)  # Remover linhas vazias
# Filtrar apenas os registros onde "Baixado" é "Não"
df_filtrado = df[df["Baixado"] == "Não"]
contador = 0
# Percorrer os registros
for _, row in df_filtrado.iterrows():
    file_name = row["Image Name"]  # Nome do arquivo
    image_url = row["Image URL"]  # URL da imagem
    img_path = os.path.join(image_folder, file_name)
    contador += 1

    if not file_name or not image_url:
        continue  # Ignorar registros inválidos

    # Caminho completo do arquivo
    file_path = os.path.join(os.getcwd(), img_path)  

    # Verificar se a imagem já existe
    if os.path.exists(file_path):
        #print(f"Imagem já existe: {file_path}")
        continue

    # Baixar a imagem
    try:
        response = requests.get(image_url, stream=True, verify=False)
        response.raise_for_status()

        # Salvar imagem
        with open(file_path, "wb") as img_file:
            for chunk in response.iter_content(chunk_size=8192):
                img_file.write(chunk)

        print(f"Imagem {contador}: {file_name}", end="\r", flush=True)  # Exibir progresso do download

        df.loc[df["Image Name"] == file_name, "Baixado"] = "Sim"  # Atualizar status no DataFrame

    except requests.exceptions.RequestException as e:
        print(f"Erro ao baixar {image_url}: {e}")
    
    if contador % 200 == 0:  # Atualizar o Excel a cada 10 downloads
        df.to_excel(excel_file, index=False)

df.to_excel(excel_file, index=False)
print("Download concluído.")    

In [ ]:
#### renomear imagens de conform de/para em arquivo Excel
import os
import pandas as pd

# Nome do arquivo Excel
excel_file = "cards_data pt1.xlsx"
pasta = "images"

# Carregar o Excel
df = pd.read_excel(excel_file, usecols=["Image Name", "novo nome", "Expansion:"])  # Lendo colunas relevantes
df.dropna(inplace=True)  # Remover linhas vazias

# Percorrer os registros
for _, row in df.iterrows():
    nome_antigo = row["Image Name"]  # Nome do arquivo
    nome_novo = row["novo nome"]  # novo nome do arquivo
    Expansion = row["Expansion:"]  # novo nome do arquivo

    Expansion = Expansion.replace("_", " ")  # Substitui espaços por "_"
    Expansion = Expansion.replace(":", " ")  # Substitui espaços por "_"
    Expansion = Expansion.replace("//", " ")  # Substitui espaços por "_"
    Expansion = Expansion.replace("  ", " ")  # Substitui ":" por "_"

    caminho_antigo = os.path.join(pasta, nome_antigo)
    
    subPasta = os.path.join(pasta, Expansion)
    os.makedirs(subPasta, exist_ok=True)
    caminho_novo = os.path.join(subPasta, nome_novo)
    # Verifica se o arquivo existe antes de renomear
    if os.path.exists(caminho_antigo):
        os.rename(caminho_antigo, caminho_novo)
        print(f'Renomeado: "{nome_antigo}" → "{nome_novo}"')
    else:
        print(f'Arquivo não encontrado: "{nome_antigo}"')
# Arborea_Pegasus_Adventures in the Forgotten Realms_530933.jpg
print("Renomeação concluída!")

In [ ]:
#### Verificar se foi baixado
import os
import pandas as pd

# Nome do arquivo Excel
excel_file = "cards_data.xlsx"
pasta = "images"

# Carregar o Excel com todas as colunas
df = pd.read_excel(excel_file)  # Lendo todo o conteúdo
contador = 0
# Verificar quais imagens estão na pasta
for _, row in df.iterrows():
    contador += 1
    baixado = row["Baixado"]  # Nome do arquiv

    # if baixado == "Sim":
    #     continue

    nome_arquivo = row["Image Name"]  # Nome do arquivo
    caminho_arquivo = os.path.join(pasta, nome_arquivo)

    if os.path.exists(caminho_arquivo):
        df.loc[df["Image Name"] == nome_arquivo, "Baixado"] = "Sim"
    else:
        df.loc[df["Image Name"] == nome_arquivo, "Baixado"] = "Não"
    
    if contador % 1000 == 0:
        print(f"Verificando {contador} imagens...", end="\r", flush=True)  # Exibir progresso da verificação
        df.to_excel(excel_file, index=False)

# Salvar mantendo todas as colunas
df.to_excel(excel_file, index=False)

print("Operação concluída!")

In [ ]:
#### Deletar imagens duplicadas
import os
import pandas as pd

# Caminho da subpasta de imagens
folder_path = "images"

# Carregar o arquivo Excel
excel_file = "cards_data pt1.xlsx"
df = pd.read_excel(excel_file)

# Obter a lista de IDs do arquivo Excel
ids = df["id"].astype(str).tolist()

# Verificar e deletar imagens duplicadas
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        # Extrair o número da imagem do nome do arquivo
        filename2 = filename.replace("_", "-")
        parts = filename2.split("-")
        if len(parts) > 1:
            image_number = parts[-1].replace(".jpg", "").strip()
            if image_number in ids:
                file_path = os.path.join(folder_path, filename)
                os.remove(file_path)
                print(f"Imagem {filename} deletada.")
            else:
                print(f"Imagem {filename} não deletada, ID não encontrado.")

print("Verificação concluída.")